In [1]:
# # Conversation Turn-Level Classification with a Pretrained BERT


In [2]:
print("hello")

hello


In [19]:
! pip install scikit-learn


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 4.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 8.0 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install pandas transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 4.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 5.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 6.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [3]:
# ============================================
# 1) Install/Import Dependencies (if needed)
# ============================================

# In a fresh environment (e.g. Google Colab), you may need:

import pandas as pd
from datasets import Dataset, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import torch


/opt/anaconda3/envs/voipvishing/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [6]:
# ============================================
# 2) Load the CSV into a Pandas DataFrame
# ============================================

# Replace with your actual CSV filename
df = pd.read_csv("/Users/ashansubodha/Desktop/VOIP Vishing/conversation-prediction/BETTER30.csv")


In [7]:
df.head()

,CONVERSATION_ID,CONVERSATION_STEP,TEXT,CONTEXT,LABEL,FEATURES,ANNOTATIONS
0,6,1,"Good morning, this is [Your Name]'s personal a...",Standard opening exchange,neutral,NaN,NaN
1,6,2,"Hello, my name is Jamie. I'm interested in vol...",Encourages the caller's interest,neutral,"welcoming, positive_tone",NaN
2,6,3,"Yes, I'm really passionate about environmental...",Reinforces anyone can volunteer,neutral,inclusive,NaN
3,6,4,"Great, how do I sign up, and where can I find ...",Demonstrates flexibility,neutral,"helpful_tone, offers_options",NaN
4,6,5,"Could you send me the link, please? And my ema...",Fulfills caller's request quickly,neutral,prompt_action,NaN


In [8]:
# ============================================
# 3) Prepare the Dataset
# ============================================

# Example: We do turn-level classification, ignoring conversation_id for now
# We'll rename columns for clarity or keep them as is

# Keep only needed columns: TEXT, LABEL
df = df[["TEXT", "LABEL"]]



In [9]:

# The model expects label as integer indices. Let's map your label strings to integers.
# For instance: "neutral" -> 0, "suspicious" -> 1, "potential_scam" -> 2, etc.

# Find unique labels
unique_labels = df["LABEL"].unique().tolist()
unique_labels.sort()  # optional, just to have them in alphabetical or consistent order
print("Unique Labels:", unique_labels)


Unique Labels: [' adhering to protocols"', ' citing urgency"', ' dismissing official protocols"', ' emphasizing security and compliance"', ' legitimate', ' neutral', ' ready for further engagement"', ' scam', ' scam_response', ' suggesting a dangerous situation"', 'Scam', 'highly_suspicious', 'legitimate', 'neutral', 'polite_ending', 'potential_scam', 'scam', 'scam_response', 'slightly_suspicious', 'standard_opening, identification_request', 'suspicious']


In [10]:

# Build a mapping dict: label -> integer
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}
print("label2id:", label2id)


label2id: {' adhering to protocols"': 0, ' citing urgency"': 1, ' dismissing official protocols"': 2, ' emphasizing security and compliance"': 3, ' legitimate': 4, ' neutral': 5, ' ready for further engagement"': 6, ' scam': 7, ' scam_response': 8, ' suggesting a dangerous situation"': 9, 'Scam': 10, 'highly_suspicious': 11, 'legitimate': 12, 'neutral': 13, 'polite_ending': 14, 'potential_scam': 15, 'scam': 16, 'scam_response': 17, 'slightly_suspicious': 18, 'standard_opening, identification_request': 19, 'suspicious': 20}


In [11]:

# Apply mapping to the dataframe
df["label_idx"] = df["LABEL"].apply(lambda x: label2id[x])

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df[["TEXT", "label_idx"]])

# Optionally, you can do a train_test_split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [12]:

print(train_dataset[0])
print(eval_dataset[0])

{'TEXT': "Hi, this is Steve from SecureNet. We're conducting a routine security check on accounts linked to your email. I need to verify some information.", 'label_idx': 16}
{'TEXT': 'Regardless of the situation, we must adhere to our verification process. We can consider supporting once we have completed our checks.', 'label_idx': 8}


In [13]:
# ============================================
# 4) Load a Pretrained Model & Tokenizer
# ============================================
# We'll use 'bert-base-uncased' as an example.
# This model already has a built-in vocabulary from its pretraining.
# So we do NOT create a custom tokenizer or vocab.

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# We'll create a model for sequence classification with the correct number of labels
num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# ============================================
# 5) Tokenize the Text
# ============================================

def tokenize_function(examples):
    return tokenizer(
        examples["TEXT"],
        padding="max_length",     # or "longest" for dynamic
        truncation=True,
        max_length=128            # adjust as necessary
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset  = eval_dataset.map(tokenize_function,  batched=True)

# Now we can remove the original TEXT to keep only the tokenized fields and label_idx
train_dataset = train_dataset.remove_columns(["TEXT"])
eval_dataset  = eval_dataset.remove_columns(["TEXT"])

# Rename label_idx to "labels" so the Trainer knows it's for classification
train_dataset = train_dataset.rename_column("label_idx", "labels")
eval_dataset  = eval_dataset.rename_column("label_idx", "labels")

# set_format transforms them into PyTorch Tensors
train_dataset.set_format("torch")
eval_dataset.set_format("torch")

print(train_dataset[0])


Map: 100%|██████████| 130/130 [00:00<00:00, 16724.73 examples/s]

{'labels': tensor(16), 'input_ids': tensor([  101,  7632,  1010,  2023,  2003,  3889,  2013,  5851,  7159,  1012,
         2057,  1005,  2128,  9283,  1037,  9410,  3036,  4638,  2006,  6115,
         5799,  2000,  2115, 10373,  1012,  1045,  2342,  2000, 20410,  2070,
         2592,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,

In [15]:
# ============================================
# 6) Define Training Arguments
# ============================================

training_args = TrainingArguments(
    output_dir="conversation-bert-model",
    evaluation_strategy="epoch",  # evaluate each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # you can tune this
    weight_decay=0.01,
    load_best_model_at_end=True,
)

/opt/anaconda3/envs/voipvishing/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# ============================================
# 7) Initialize Trainer
# ============================================

def compute_metrics(eval_pred):
    """
    Optionally define a function to compute custom metrics.
    eval_pred is (predictions, labels).
    For simplicity, let's just compute accuracy.
    """
    from sklearn.metrics import accuracy_score
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,   # not strictly required if we're already tokenized
    compute_metrics=compute_metrics
)

/var/folders/2t/7vrwz9ms6c1b8_81nddnvzkr0000gn/T/ipykernel_3144/3257189120.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
# ============================================
# 8) Train the Model
# ============================================

trainer.train()

                                                 
 33%|███▎      | 130/390 [00:31<00:58,  4.47it/s]

{'eval_loss': 1.50783109664917, 'eval_accuracy': 0.5307692307692308, 'eval_runtime': 1.8462, 'eval_samples_per_second': 70.417, 'eval_steps_per_second': 17.875, 'epoch': 1.0}


                                                 
 67%|██████▋   | 260/390 [01:04<00:29,  4.48it/s]

{'eval_loss': 1.2062383890151978, 'eval_accuracy': 0.6153846153846154, 'eval_runtime': 1.7585, 'eval_samples_per_second': 73.927, 'eval_steps_per_second': 18.766, 'epoch': 2.0}


                                                 
100%|██████████| 390/390 [01:37<00:00,  4.46it/s]

{'eval_loss': 1.1580626964569092, 'eval_accuracy': 0.6076923076923076, 'eval_runtime': 1.7853, 'eval_samples_per_second': 72.816, 'eval_steps_per_second': 18.484, 'epoch': 3.0}


100%|██████████| 390/390 [01:39<00:00,  3.93it/s]

{'train_runtime': 99.3207, 'train_samples_per_second': 15.707, 'train_steps_per_second': 3.927, 'train_loss': 1.205150349934896, 'epoch': 3.0}


TrainOutput(global_step=390, training_loss=1.205150349934896, metrics={'train_runtime': 99.3207, 'train_samples_per_second': 15.707, 'train_steps_per_second': 3.927, 'total_flos': 102630816737280.0, 'train_loss': 1.205150349934896, 'epoch': 3.0})

In [19]:
# ============================================
# 10) Inference / Prediction
# ============================================
# Suppose we want to predict the label for a new text sample

sample_text = "Good Morning"
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, max_length=128)
inputs = {k: v.to(trainer.model.device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax().item()

predicted_label = id2label[predicted_class_id]
print(f"Sample Text: {sample_text}")
print(f"Predicted Label: {predicted_label}")


Sample Text: Good Morning
Predicted Label:  neutral
